In [5]:
import cirq
import math

# 量子スパイハンターを体験するためのハンズオン
# 量子鍵配送のために考案されたアルゴリズムです。

# メイン関数
def main():
    # qcインスタンス生成
    qc = QPU()
    # キュビットを3つ割り当てる。
    qc.reset(3)
    a = 0x1
    fiber = 0x2
    b = 0x4

    # ２つの秘密のキュビットを生成する。
    send_had = random_bit()
    send_value = random_bit()

    # アリスのスパイハンターキュビットを用意する。
    if send_value:
        # 値を設定
        qc.x(a)
    if send_had:
        # had命令を使って値を設定する/しない
        qc.had(a)

    # キュビットを送信します!
    qc.exchange(fiber, a)

    # スパイハンターを有効化させます！
    spy_is_present = True
    
    if (spy_is_present):
        spy_had = 1
        if spy_had:
            qc.had(fiber)
        qc.read(fiber, 'stolen_data')

    # キュビットを受け取る！
    recv_had = random_bit()
    #キュビットを送信!
    qc.exchange(fiber, b)
    
    if recv_had:
        # had命令を適用する/しない
        qc.had(b)
    #観測する
    qc.read(b, 'recv_val')

    qc.draw() 
    # 結果を取得する。
    result = qc.run() 
    print(result)
    # 結果
    recv_val = 1 if result.measurements['recv_val'][0] else 0

    # had命令が適用されていることを確認する。
    if (send_had == recv_had):
        # 両者の保有する値が一致することを確認する。
        if (send_value != recv_val):
            # 出力する。
            print('Caught a spy!\n')

# ランダムビットを生成するための関数
def random_bit():
    # キュビット生成
    rng = QPU()
    # キュビットに１を割り当てる。
    rng.reset(1)
    # 重ね合わせ配置
    rng.had()
    # 観測
    rng.read()
    result = rng.run()
    # ランダムビット生成
    bit = 1 if result.measurements['result'][0] else 0
    # bitを返す。
    return bit


######################################################################
## QPU命令を使用するためのインターフェース群
class QPU:
    def __init__(self):
        self.circuit = cirq.Circuit()
        self.simulator = cirq.Simulator()
        self.qubits = None

    def reset(self, num_qubits):
        self.qubits = [cirq.GridQubit(i, 0) for i in range(num_qubits)]

    def mask_to_list(self, mask):
        return [q for i,q in enumerate(self.qubits) if (1 << i) & mask]

    def had(self, target_mask=~0):
        target = self.mask_to_list(target_mask)
        self.circuit.append(cirq.H.on_each(*target))

    def x(self, target_mask=~0):
        target = self.mask_to_list(target_mask)
        self.circuit.append(cirq.X.on_each(*target))

    def cnot(self, target_mask, control_mask):
        target = self.mask_to_list(target_mask)
        control = self.mask_to_list(control_mask)
        self.circuit.append(cirq.CNOT.on(control[0], target[0]))

    def exchange(self, q0_mask, q1_mask):
        # Construct SWAP per Figure 3-21 in the book
        self.cnot(q0_mask, q1_mask)
        self.cnot(q1_mask, q0_mask)
        self.cnot(q0_mask, q1_mask)

    def phase(self, theta_degrees, target_mask=~0):
        target = self.mask_to_list(target_mask)
        theta_radians = theta_degrees * math.pi / 180.0
        self.circuit.append(cirq.Rz(theta_radians).on_each(*target))

    def rootnot(self, target_mask=~0):
        sqrt_x = cirq.X**0.5
        target = self.mask_to_list(target_mask)
        self.circuit.append(sqrt_x.on_each(*target))

    def read(self, target_mask=~0, key=None):
        if key is None:
            key = 'result'
        target = self.mask_to_list(target_mask)
        self.circuit.append(cirq.measure(*target, key=key))

    def draw(self):
        print('Circuit:\n{}'.format(self.circuit))

    def run(self):
        return self.simulator.simulate(self.circuit)


if __name__ == '__main__':
    main()

Circuit:
(0, 0): ───X───H───@───X───@──────────────────────────────────────────────────────────
                   │   │   │
(1, 0): ───────────X───@───X───H───M('stolen_data')───X───@───X───────────────────────
                                                      │   │   │
(2, 0): ──────────────────────────────────────────────@───X───@───H───M('recv_val')───
measurements: recv_val=0 stolen_data=1
output vector: |000⟩
Caught a spy!

